In [1]:
#Cite from: https://github.com/simonmeister/pytorch-mono-depth
import os 
import os.path as osp
import numpy as np
import torch as th
import torch.nn as nn
from math import log
from matplotlib import pyplot as plt 
import sys 
import pandas as pd 
from tqdm import tqdm

from torch.utils.data import DataLoader
sys.path.append("../src")
from depth_model.model import PTModel
from data_loader.data_creation import CreateDataset
from utils.data_transforms import pre_process

print(th.cuda.is_available())
device = 'cuda' if th.cuda.is_available() else 'cpu'

True


### Define Losses 


In [2]:
def _mask_input(input, mask=None):
    if mask is not None:
        input = input * mask
        count = th.sum(mask).data[0]
    else:
        count = np.prod(input.size(), dtype=np.float32).item()
    return input, count

class BerHuLoss(nn.Module):
    def forward(self, input, target, mask=None):
        x = input - target
        abs_x = th.abs(x)
        c = th.max(abs_x).item() / 5
        leq = (abs_x <= c).float()
        l2_losses = (x ** 2 + c ** 2) / (2 * c)
        losses = leq * abs_x + (1 - leq) * l2_losses
        losses, count = _mask_input(losses, mask)
        return th.sum(losses) / count

class HuberLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.loss = nn.SmoothL1Loss(size_average=False)

    def forward(self, input, target, mask=None):
        if mask is not None:
            loss = self.loss(input * mask, target * mask)
            count = th.sum(mask).data[0]
            return loss / count

        count = np.prod(input.size(), dtype=np.float32).item()
        return self.loss(input, target) / count

class DistributionLogLoss(nn.Module):
    def __init__(self, distribution):
        super().__init__()
        self.distribution = distribution

    def forward(self, input, target, mask=None):
        d = self.distribution(*input)
        loss = d.log_loss(target)
        loss, count = _mask_input(loss, mask)
        return th.sum(loss) / count
    

class RMSLoss(nn.Module):
    def forward(self, input, target, mask=None):
        loss = th.pow(input - target, 2)
        loss, count = _mask_input(loss, mask)
        return th.sqrt(th.sum(loss) / count)
    

class RelLoss(nn.Module):
    def forward(self, input, target, mask=None):
        loss = th.abs(input - target) / target
        loss, count = _mask_input(loss, mask)
        return th.sum(loss) / count
    
class MseLoss(nn.Module):  
    def forward(self, input, target, mask=None):
        loss = th.sum((input - target) ** 2)
        loss, count = _mask_input(loss, mask)
        return th.sum(loss) / count

## Load Model 

In [3]:
depth_model = PTModel().to(device)
depth_model.load_state_dict(th.load("../model/child_depth_model.ckpt"))

/home/samartht/anaconda3/envs/pydepth/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/samartht/anaconda3/envs/pydepth/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

## Run Inference and get metric on Validation Images 

In [4]:
criterion  = BerHuLoss()
criterion2 = RelLoss() 
criterion3 = RMSLoss()

def validate(model, test_loader,batch_size = 1):
    model.eval()
    with th.no_grad():
      loss2, loss ,loss3 = 0.0, 0.0, 0.0
      for sample_batch in tqdm(test_loader,total=len(test_loader)):
            t_image = sample_batch['image']
            t_depth = sample_batch['depth']
            
            t_image = t_image.to(device)
            t_depth = t_depth.to(device)
            t_outputs = model(t_image)
            
            curr_loss = criterion(t_depth, t_outputs) ## BerHuLoss
            curr_loss2 = criterion2(t_depth, t_outputs) ## REL Loss
            curr_loss3 = criterion3(t_depth,t_outputs) ## RMS Loss
            loss += curr_loss.item()
            loss2 += curr_loss2.item()
            loss3 += curr_loss3.item()
            
      print("Validation BerHuLoss: {:.4f}"
            .format(loss/(len(test_loader) * batch_size)))
      print("Validation REL LOSS: {:.4f}"
            .format(loss2/(len(test_loader) * batch_size)))
      print("Validation RMSE: {:.4f}"
            .format(loss3/(len(test_loader) * batch_size)))

### Run Evaluation on Assignment data 

In [5]:
from glob import glob 
import sys 
sys.path.append("../src/")

from  utils.load_tof_images import create_from_zip_absolute  as load_assignment_data
from data_loader.data_loader_assignment import CreateAssignemntDataset
from utils.data_transforms import pre_process

print(device)

cuda


In [7]:
path = "../data/360_scan/"
child_rgb_files = glob(path+"/*/rgb/*")
child_depth_files = glob(path+'/*/depth/*')

print("Total RGB images ",len(child_rgb_files))
print("Total Depth Images ",len(child_depth_files))

batch_size = 1
assignment_loader = CreateAssignemntDataset(child_rgb_files,child_depth_files,process_image=pre_process())
valid_loader = DataLoader(assignment_loader, batch_size=1, shuffle=False, num_workers=1)

Total RGB images  255
Total Depth Images  255


In [8]:
validate(depth_model, valid_loader)

  0%|          | 0/255 [00:00<?, ?it/s]/mnt/e/Personal/Samarth/repository/Depth-Estimation/notebooks/../src/utils/data_transforms.py:43: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  img = th.from_numpy(np.transpose(pic, (2, 0, 1)))
100%|██████████| 255/255 [01:36<00:00,  2.64it/s]

Validation BerHuLoss: 0.2191
Validation REL LOSS: 0.0208
Validation RMSE: 0.4057
